# Student Performance Analytics System - ETL Pipeline

This notebook demonstrates the ETL (Extract, Transform, Load) pipeline for the Student Performance Analytics System project. It ingests raw CSV data, loads it into a SQLite staging area, and performs SQL transformations to create a Star Schema Data Warehouse.

In [ ]:
import pandas as pd
import sqlite3
import os
import glob

# Configuration
# Since this notebook is in the 'notebooks' directory, we go up one level to the project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
DB_PATH = os.path.join(DATA_DIR, 'student_dwh.db')
SQL_SCRIPT_DIR = os.path.join(PROJECT_ROOT, 'sql')

print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Directory: {DATA_DIR}")
print(f"Database Path: {DB_PATH}")

## 1. Database Connection & Initialization
We define a helper function to connect to the SQLite database and initialize the schema using the DDL script.

In [ ]:
def get_db_connection():
    return sqlite3.connect(DB_PATH)

def init_db():
    """Initializes the database with DDL scripts if not exists."""
    print("Initializing Database...")
    conn = get_db_connection()
    try:
        with open(os.path.join(SQL_SCRIPT_DIR, '1_ddl_schema.sql'), 'r') as f:
            conn.executescript(f.read())
        print("Database Schema Created.")
    except Exception as e:
        print(f"Error initializing database: {e}")
    finally:
        conn.close()

# Run Initialization
init_db()

## 2. Load Data to Staging (ETL)
This step reads CSV files from the `data` directory, cleans them (handling dates), and loads them into staging tables in SQLite.

In [ ]:
def load_data_to_staging():
    """
    ETL Step: Reads CSVs from Data Folder, cleans them, and loads to Staging tables.
    """
    conn = get_db_connection()
    
    # Map filenames (partial match) to table names and date columns
    file_mappings = {
        'students': {'table': 'stg_students', 'date_cols': ['Date_of_Birth']},
        'attendance': {'table': 'stg_attendance', 'date_cols': ['Date']},
        'performance': {'table': 'stg_performance', 'date_cols': []},
        'homework': {'table': 'stg_homework', 'date_cols': ['Due_Date']},
        'communication': {'table': 'stg_communication', 'date_cols': ['Date']}
    }

    print(f"\nScanning for data in: {DATA_DIR}")
    
    for key, config in file_mappings.items():
        # Find file matching the key
        files = glob.glob(os.path.join(DATA_DIR, f"*{key}*.csv"))
        
        if not files:
            print(f"Warning: No file found for {key}")
            continue
            
        file_path = files[0] # Take the first match
        print(f"Loading {os.path.basename(file_path)} -> {config['table']}...")
        
        try:
            df = pd.read_csv(file_path)
            
            # 1. Data Cleaning (Pandas)
            # Handle Date Columns
            for date_col in config['date_cols']:
                if date_col in df.columns:
                    df[date_col] = pd.to_datetime(df[date_col], errors='coerce').dt.date
            
            # Special handling for Performance table
            if key == 'performance' and 'Homework_Completion_%' in df.columns:
                df.rename(columns={'Homework_Completion_%': 'Homework_Completion_Pct'}, inplace=True)
                
            # 2. Load to SQLite (Staging)
            # if_exists='replace' ensures we start fresh each run for this demo
            df.to_sql(config['table'], conn, if_exists='replace', index=False)
            print(f"  -> Loaded {len(df)} rows.")
            
        except Exception as e:
            print(f"  -> Error loading {key}: {e}")
            
    conn.close()

# Run ETL
load_data_to_staging()

## 3. Run ELT Transformations
This step runs the SQL scripts to transform the data in the staging tables into the final Fact and Dimension tables.

In [ ]:
def run_elt_transformations():
    """
    ELT Step: Runs SQL scripts to transform Staging data into Fact/Dimension tables.
    """
    print("\nRunning ELT Transformations (SQL)...")
    conn = get_db_connection()
    
    try:
        with open(os.path.join(SQL_SCRIPT_DIR, '2_views_and_procedures.sql'), 'r') as f:
            conn.executescript(f.read())
        print("  -> Transformations Complete. Data Warehouse is ready.")
    except Exception as e:
        print(f"  -> Error in SQL Transformations: {e}")
    finally:
        conn.close()

# Run ELT
run_elt_transformations()

## 4. Verification
Let's query the database to verify that the data has been loaded and transformed correctly.

In [ ]:
def verify_data():
    conn = get_db_connection()
    try:
        # Check a few tables
        tables = ['Dim_Student', 'Fact_Performance']
        for table in tables:
            count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
            print(f"Table {table} has {count} rows.")
            
        # Sample query
        print("\nSample Data from Fact_Performance:")
        df = pd.read_sql_query("SELECT * FROM Fact_Performance LIMIT 5", conn)
        print(df)
    except Exception as e:
        print(f"Verification failed: {e}")
    finally:
        conn.close()

verify_data()